In [7]:
!pip -q install langchain
!pip -q install guardrails-ai

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import getpass

from rich import print

from langchain.output_parsers import GuardrailsOutputParser

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

OPENAI_API_KEY = getpass.getpass()

In [5]:
input_problem = input("Enter the problem: ")
input_nodes = input("Enter the nodes: ")
input_iterations = input("Enter the iterations: ")

In [ ]:
chat = ChatOpenAI(temperature=0.9 , model="gpt-3.5-turbo-16k", client=any , openai_api_key=OPENAI_API_KEY)

formated_problem = {
  "problem": "problem",
  "approach": [
    {
      "approach": "",
      "score": ""
    },
    {
      "approach": "",
      "score": ""
    },
    {
      "approach": "",
      "score": ""
    }
  ]
}


messages = [
    SystemMessage(
        content="You are an AI who is really good at solving problems. You always think step by step and come up with different approaches to solve a problem",
    ),
    HumanMessage(
        content=
        f"""
        Given this problem: {input_problem}
        let solve the problem step by step.
        you have {input_iterations} iterations to solve it.
        come up with {input_nodes} different approaches to solve it.
        
        give each approach a score from 0 to 10 based on how good it is.
        
        give it to me in this format:
        {str(formated_problem)}
        """
    ),
]
step1 = chat(messages).content

In [ ]:
extractor_rail = """
<rail version="0.1">

<output>
    <string name="problem" description="problem specified" />
    <list name="approaches" description="List of different approaches to solve the problem">
        <object>
            <string name="approach" description="the approach"/>
            <integer name="score" format="valid-range: 0 10" />
        </object>
    </list>
</output>

<prompt>

given the following problem and steps to solve the problem, please extract the problem and a list of different approaches 

{{approach}}

@complete_json_suffix_v2
</prompt>
</rail>
"""

output_parser = GuardrailsOutputParser.from_rail_string(extractor_rail)

prompt = PromptTemplate(
    template=output_parser.guard.base_prompt,
    input_variables=output_parser.guard.prompt.variable_names,
)

model = OpenAI(temperature=0 , openai_api_key=OPENAI_API_KEY) # type: ignore
# model = OpenAI(temperature=0)
print(step1)
solution = model(prompt.format_prompt(approach=step1).to_string())
solution = output_parser.parse(solution)
print(solution)